In [4]:
!pip install -q diffusers transformers torch moviepy pillow

In [5]:
import torch
from diffusers import StableDiffusionPipeline
from moviepy.editor import ImageSequenceClip
from PIL import Image
import numpy as np
import os
import time
from IPython.display import HTML
from google.colab import files

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [6]:
def create_animation_from_prompt(
    prompt,
    num_frames=15,
    fps=5,
    guidance_scale=7.5,
    height=512,
    width=512,
    seed=None,
    animation_type="zoom",
    motion_strength=0.05
):
    """
    Create a simple animation video based on a text prompt.

    Parameters:
    - prompt (str): The text description for generating the animation
    - num_frames (int): Number of frames to generate (maximum 15)
    - fps (int): Frames per second in the output video
    - guidance_scale (float): How strongly the generation follows the prompt
    - height, width (int): Dimensions of the output video
    - seed (int): Random seed for reproducibility (None for random)
    - animation_type (str): Type of animation effect ('zoom', 'pan', 'morph')
    - motion_strength (float): How strong the animation effect should be

    Returns:
    - str: Path to the generated video file
    """
    # Ensure we don't exceed 15 frames
    num_frames = min(num_frames, 15)

    print(f"Generating animation with {num_frames} frames based on prompt: '{prompt}'")

    # Initialize stable diffusion pipeline
    pipe = StableDiffusionPipeline.from_pretrained(
        "runwayml/stable-diffusion-v1-5",
        torch_dtype=torch.float16,
        safety_checker=None
    )

    # Move to GPU if available
    device = "cuda" if torch.cuda.is_available() else "cpu"
    pipe = pipe.to(device)

    # Set random seed if provided
    generator = None
    if seed is not None:
        generator = torch.Generator(device=device).manual_seed(seed)

    # Create output directory
    os.makedirs("animation_frames", exist_ok=True)

    # Generate frames
    frames = []
    frame_paths = []

    for i in range(num_frames):
        # Modify prompt slightly for each frame to create motion
        current_prompt = prompt

        # Apply different animation techniques
        if animation_type == "morph":
            # For morphing, subtly change the prompt descriptors
            if i > 0:
                variation = int((i / num_frames) * 100)
                current_prompt = f"{prompt}, {variation}% variation"

        # For zoom and pan, we'll use the same prompt but modify the noise
        noise_offset = i * motion_strength if animation_type != "morph" else 0

        # Set a different seed for each frame but with controlled variation
        frame_seed = seed + i if seed is not None else None
        frame_generator = None if frame_seed is None else torch.Generator(device=device).manual_seed(frame_seed)

        # Generate the image
        print(f"Generating frame {i+1}/{num_frames}...")

        # Add slight variation to guidance scale for subtle changes
        current_guidance = guidance_scale * (1 + (i * 0.01))

        image = pipe(
            current_prompt,
            num_inference_steps=30,
            guidance_scale=current_guidance,
            generator=frame_generator,
            height=height,
            width=width
        ).images[0]

        # Save the frame
        frame_path = f"animation_frames/frame_{i:03d}.png"
        image.save(frame_path)
        frame_paths.append(frame_path)

        # Convert to numpy array for moviepy
        frame_array = np.array(image)
        frames.append(frame_array)

    # Create video clip
    clip = ImageSequenceClip(frames, fps=fps)
    output_path = "prompt_animation.mp4"
    clip.write_videofile(output_path, codec="libx264")

    print(f"Animation created! Video saved to {output_path}")

    # Download the video
    files.download(output_path)

    # Display in the notebook
    return HTML(f"""
    <video width="512" height="512" controls>
      <source src="file://{output_path}" type="video/mp4">
    </video>
    """)


In [9]:
def generate_animation(prompt):
    # Get user prompt
    user_prompt = prompt

    # Optional configuration
    num_frames = 15

    animation_type = "morph"
    # Generate the animation
    return create_animation_from_prompt(
        prompt=user_prompt,
        num_frames=num_frames,
        animation_type=animation_type,
        fps=5
    )

In [12]:
prompt = "horse running in race"
generate_animation(prompt)

Generating animation with 15 frames based on prompt: 'horse running in race'


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


Generating frame 1/15...


  0%|          | 0/30 [00:00<?, ?it/s]

...

  0%|          | 0/30 [00:00<?, ?it/s]

Moviepy - Building video prompt_animation.mp4.
Moviepy - Writing video prompt_animation.mp4



Moviepy - Done !
Moviepy - video ready prompt_animation.mp4
Animation created! Video saved to prompt_animation.mp4


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>